In [76]:
%cd ~/google_drive/code/projects/sparse/python
from sparse.core.sparse_dataframe import SparseDataFrame
from sparse.utilities.utils import *

%cd ~/google_drive/code/projects/texture_classifier/python
# %cd /home/ubuntu/texture_classifier/python
import multiprocessing
import PIL
from sklearn.cluster import KMeans
from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist, squareform
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
import cv2
from pandas.io.pytables import HDFStore
# from plotly import plotly
# from plotly.graph_objs import *

import core.utils
reload(core.utils)
from core.utils import *

import core.image_scanner
reload(core.image_scanner)
from core.image_scanner import ImageScanner

import core.pipeline
reload(core.pipeline)
from core.pipeline import *

/Users/alexbraun/google_drive/code/projects/sparse/python
/Users/alexbraun/google_drive/code/projects/texture_classifier/python


In [ ]:
def archive(info, hdf_path, features=['r', 'g', 'b', 'h', 's', 'v', 'fft_max', 'fft_std']):
    hdf = HDFStore(hdf_path)
    data = get_data(info, features=features)
    x = data.drop('y', axis=1)
    y = data.y
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2)
    hdf['train_x'] = DataFrame(train_x)
    hdf['test_x'] = DataFrame(test_x)
    hdf['train_y'] = Series(train_y)
    hdf['test_y'] = Series(test_y)
    hdf.close()
    return train_x, test_x, train_y, test_y

In [72]:
source = '/Users/alexbraun/Documents/data/texture_classifier/data/texture'
spec = [
    'material',
    'image_id',
    'label',
    'origin',
    'desc',
    'extension'
]

wood_mask = [
    'olive',
    'european-ash',
    'african-mahogany',
    'bamboo',
    'sapele',
    'kingwood',
    'teak'
#     'european-lime',
#     'makore'
]

In [73]:
# # random scan
# min_res = 10
# max_res = 100
# params = {
#             'scan_method':      'random_scan',
#             'min_resolution':   (min_res, min_res),
#             'max_resolution':   (max_res, max_res),
#             'patches':          50,
#             'patch_resolution': (min_res, min_res),
# #             'normalize':        True,
#             'bins':             256
# #             'rotation':         'random'
# }

# # grid scan
min_res = 100
max_res = 100
params = {
            'scan_method':      'grid_scan',
            'min_resolution':   (min_res, min_res),
            'max_resolution':   (max_res, max_res),
            'resolutions':      1,
            'spacing':          'even',
#             'patch_resolution': (min_res, min_res),
#             'normalize':        True,
            'bins':             256
}

info = get_info(source, spec)
# info = info[info.common_name.apply(lambda x: x in wood_mask)]

# dataframes won't allow direct assignment of dicts
info['params'] = None
info.params = info.params.apply(lambda x: params)

# info = info[(info.desc.apply(lambda x: '_d_' in x)) | (info.origin != 'arroway-textures')]

# info = info[info.desc.apply(lambda x: '_a_' in x)]
# info.common_name = info.common_name.apply(lambda x: re.sub('--.*', '', x))

# train, test = info_split(info)

# train_info = info[info.desc.apply(lambda x: '_d_' in x)]
# test_info = info[info.origin == 'wood-database']i

# train_x, test_x, train_y, test_y = archive(info, hdf_path)

# info = info[info.origin != 'arroway-textures']
# info = info.head(20)

In [ ]:
# %%snakeviz

version = '10'.zfill(3)
hdf_path = '/Us00ers/alexbraun/Documents/data/texture_classifier/data/hdf/data.'
hdf_path += version + '.aw-only.grid.hdf'

# train_x, valid_x, test_x, train_y, valid_y, test_y = archive_data(train_info, test_info, hdf_path)
# %time train_x, test_x, train_y, test_y = archive_data(train, test, hdf_path=hdf_path, cross_val=False)
   
# train_x, valid_x, test_x, train_y, valid_y, test_y = read_archive(hdf_path)

# data = _batch_get_data(info)
data = get_data(info, hdf_path, processes=20)

In [ ]:
clf = SVC()
params = {
    'C':            [1], #np.arange(0.1, 1, 0.1),
    'kernel':       ['linear', 'rbf'],
#     'degree':       [3],
#     'gamma':        [0.0],
#     'coef0':        [0.0],
#     'shrinking':    [True],
#     'probability':  [False],
#     'tol':          [0.001],
#     'cache_size':   [200],
#     'class_weight': [None],
#     'verbose':      [False],
    'max_iter':     [1],
#     'random_state': [None]
}

grid = GridSearchCV(clf, params, cv=5)
%time grid.fit(train_x, train_y)
print(grid.best_params_)
# pred = grid.best_estimator_.predict(valid_x)
# valid_report = get_report(valid_y, pred)
# valid_report
pred = grid.best_estimator_.predict(test_x)
print(grid.best_estimator_.score(test_x, test_y))
report = get_report(test_y, pred)
report

In [ ]:
clf = RandomForestClassifier()
params = {
#     'n_estimators':        range(3, 20), #[10, 43, 100],
#     'criterion':           ['gini'],
#     'max_depth':           [None],
#     'min_samples_split':   [2],
#     'min_samples_leaf':    [1],
#     'max_features':        ['auto', 100, 500, 1000],
#     'max_leaf_nodes':      [None],
#     'bootstrap':           [True],
#     'oob_score':           [False],
    'n_jobs':              [-1]
#     'random_state':        [42]
#     'verbose':             [0],
#     'min_density':         [None],
#     'compute_importances': [None]
}

grid = GridSearchCV(clf, params, cv=5)%time grid.fit(train_x, train_y)
print(grid.best_params_)
# pred = grid.best_estimator_.predict(valid_x)
# valid_report = get_report(valid_y, pred)
# valid_report
pred = grid.best_estimator_.predict(test_x)
print(grid.best_estimator_.score(test_x, test_y))
report = get_report(test_y, pred)
report

In [ ]:
def majority_vote(y_true, y_pred):
    d = DataFrame([y_true, y_pred]).T
    d.columns = ['ytrue', 'yhat']
    lut = {k:i for i, k in enumerate(wood_mask)}
    ilut = {v:k for k, v in lut.iteritems()}
    d.yhat = d.yhat.apply(lambda x: lut[x])
    d = d.groupby('ytrue').agg(lambda x: x.mode()).yhat.apply(lambda x: ilut[x])
    d = DataFrame(d)
    print(d[d.index == d.yhat].shape[0] / float(d.shape[0]))
    return d

def predict(info, estimator):
    y = []
    for i, row in info.iterrows():
        x = get_data(row.to_frame().T)
        x = x.drop('y', axis=1)
        x = Series(estimator.predict(x)).mode().values[0]
        y.append(x)
    return Series(y)

# majority_vote(test_y, pred)

In [ ]:
pred = predict(info, grid.best_estimator_)
get_report(info.common_name, pred)

In [84]:
pred = ['abura'] * 3 + ['box-elder'] * 2 + ['banana'] * 4

def get_results(pred, info):
    data = pred.merge(info, how='left', on='label')
#     data.drop_duplicates('label', inplace=True)
#     data = data.apply(lambda x: x.to_dict(), axis=1).tolist()
    return data

pred = compile_predictions(pred)
get_results(pred, info)

,confidence,label,material,image_id,origin,desc,extension,source,params
0,0,abura,wood,00000,wood-database,#d_sanded,jpg,/Users/alexbraun/Documents...,"{u'spacing': u'even', u'ma..."
1,0,abura,wood,00000,wood-database,#d_sanded_sealed,jpg,/Users/alexbraun/Documents...,"{u'spacing': u'even', u'ma..."
2,0,banana,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,box-elder,wood,00296,wood-database,#d_,jpg,/Users/alexbraun/Documents...,"{u'spacing': u'even', u'ma..."
4,0,box-elder,wood,00297,wood-database,#d_sealed,jpg,/Users/alexbraun/Documents...,"{u'spacing': u'even', u'ma..."
